In [1]:
import torch
from torch import nn
from torch import optim
import numpy as np
import time
import matplotlib.pyplot as plt
import os

## Configuration

In [2]:
# Number of force cells in the robotic leg
N_CELLS = 8

# Path where the results are stored
RESULTS_PATH = '../../../../results'
# ID of the training and validation data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0006_14062021'
# Number of folds for cross-validation
CV = 6

print('Model training with data: ' + DATA_ID)

TORCH_DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', TORCH_DEVICE)

Model training with data: 0006_14062021
Using device: cuda


## Recurrent Neural Network

In [22]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_dim, sigma, output_size, sequence_length, num_data_train, num_iter, n_layers, device, lr=0.001, batch_size=32):
        
        # input size -> Dimension of the input signal
        # outpusize -> Dimension of the output signal
        # hidden_dim -> Dimension of the rnn state
        # n_layers -> If >1, we are using a stacked RNN
        
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.input_size = input_size
        self.sigma = sigma
        self.output_size = output_size
        self.sequence_length = sequence_length
        self.num_layers = n_layers
        
        # define an RNN with specified parameters
        # batch_first=True means that the first dimension of the input will be the batch_size
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_dim, num_layers=n_layers, 
                          nonlinearity='relu', batch_first=True)
        
        # last, fully-connected layer
        self.fc1 = nn.Linear(hidden_dim, output_size) # YOUR CODE HERE

        
        self.lr = lr #Learning Rate
        self.batch_size = batch_size
        self.num_train = num_data_train #Number of training signals
        self.optim = optim.Adam(self.parameters(), self.lr)
        self.num_iter = num_iter
        self.criterion = nn.MSELoss() #YOUR CODE HERE     
        
        self.device = device
        self.to(self.device)
        
        # A list to store the loss evolution along training
        self.loss_during_training = [] 
        
    def forward(self, x, h0=None):
        
        '''
        About the shape of the different tensors ...:
        
        - Input signal x has shape (batch_size, seq_length, input_size)
        - The initialization of the RNN hidden state h0 has shape (n_layers, batch_size, hidden_dim).
          If None value is used, internally it is initialized to zeros.
        - The RNN output (batch_size, seq_length, hidden_size). This output is the RNN state along time  

        '''
        
        batch_size = x.size(0) # Number of signals N
        seq_length = x.size(1) # T
        
        # get RNN outputs
        # r_out is the sequence of states
        # hidden is just the last state (we will use it for forecasting)
        print(x.shape)
        r_out, hidden = self.rnn(x, h0)
        print(r_out.shape, hidden.shape)
        
        # shape r_out to be (seq_length, hidden_dim) #UNDERSTANDING THIS POINT IS IMPORTANT!!        
        r_out = r_out.reshape(-1, self.hidden_dim) 
        print(r_out.shape)
        
        output = self.fc1(r_out)
        print(output.shape)
        
        noise = torch.randn_like(output) * self.sigma
        
        output += noise
        
        # reshape back to temporal structure
        output = output.reshape([-1, self.sequence_length, 1])

        return output, hidden
           
    def train(self,x,y):
        
        # SGD Loop
        splits = [i for i in range(0, x.shape[0], self.sequence_length)]
        
        for e in range(int(self.num_iter)):
        
            running_loss = 0.
            
            for idx, i in enumerate(splits[:-1]):
                features = x[np.newaxis, splits[idx]:splits[idx + 1], :]
                target = y[splits[idx]:splits[idx + 1], np.newaxis]
                print(features.shape, target.shape)
                self.optim.zero_grad() 
                
                features = torch.Tensor(features).to(self.device) #YOUR CODE HERE  
                target = torch.Tensor(target).to(self.device) #YOUR CODE HERE  
                
                out,hid = self.forward(features)

                print(out, target)
                loss = self.criterion(out[-1], target[-1]) #YOUR CODE HERE
                running_loss += loss.item()

                loss.backward()

                # This code helps to avoid vanishing exploiting gradients in RNNs
                # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
                nn.utils.clip_grad_norm_(self.parameters(), 2.0)

                self.optim.step()
                
            self.loss_during_training.append(running_loss / len(splits))
            
            if(e % 1 == 0): # Every 1 iteration

                print("Iteration %d. Training loss: %f" %(e,self.loss_during_training[-1]))        

In [4]:
# Load data
fold_id = 1
X_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_train_cv{}_{}.npy'.format(fold_id + 1, DATA_ID)))
X_valid = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_valid_cv{}_{}.npy'.format(fold_id + 1, DATA_ID)))
Y_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_train_cv{}_{}.npy'.format(fold_id + 1, DATA_ID)))
Y_valid = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_valid_cv{}_{}.npy'.format(fold_id + 1, DATA_ID)))

print(X_train.shape, Y_train.shape)
print(X_valid.shape, Y_valid.shape)

(385544, 4) (385544, 24)
(72610, 4) (72610, 24)


In [ ]:
X_train -> [batch_size=8, sequence_length=50, features=4]
Y_train -> [batch_size=8, sequence_length=1, target=1]

In [23]:
model = RNN(input_size=4, hidden_dim=32, sigma=1, output_size=1, sequence_length=50, 
            num_data_train=X_train.shape[0], num_iter=200, n_layers=1, device=TORCH_DEVICE)

In [24]:
force_cell_col = 9

model.train(X_train, Y_train[:, force_cell_col])

(1, 50, 4) (50, 1)
torch.Size([1, 50, 4])
torch.Size([1, 50, 32]) torch.Size([1, 1, 32])
torch.Size([50, 32])
torch.Size([50, 1])
tensor([[[ 1.1078],
         [-0.6078],
         [ 1.6880],
         [-0.0706],
         [ 0.6010],
         [-1.9157],
         [ 0.5240],
         [ 1.0362],
         [ 0.9187],
         [ 0.4437],
         [-0.8499],
         [-1.6744],
         [-0.6291],
         [ 2.6949],
         [ 1.1898],
         [ 0.3279],
         [-1.5810],
         [-0.6678],
         [ 0.7024],
         [ 0.5841],
         [-0.9361],
         [-1.6673],
         [-0.9618],
         [ 0.2881],
         [-0.6574],
         [ 0.2859],
         [ 0.1514],
         [ 0.6040],
         [ 0.5602],
         [ 0.7805],
         [-2.2959],
         [ 0.1421],
         [ 0.7086],
         [ 0.5460],
         [ 0.4377],
         [ 0.4473],
         [-0.4870],
         [ 0.3946],
         [ 0.3736],
         [ 1.3519],
         [-0.0766],
         [ 0.5235],
         [-1.8135],
         [

KeyboardInterrupt: 